In [1]:

import tensorflow


In [2]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

### Load the dataset

As we are using google colab, we need to mount the google drive to load the data file

In [3]:
# IMPORT DATA
import pandas as pd
import numpy as np

data = pd.read_csv('ner_dataset.csv', encoding='latin1')
data = data.fillna(method="ffill") # Deal with N/A

In [4]:
tags = list(set(data["POS"].values)) # Read POS values

In [5]:
tags # List of possible POS values

['VBD',
 'RBR',
 'WRB',
 'TO',
 'NNPS',
 'CD',
 '``',
 'IN',
 'JJR',
 'DT',
 'WP',
 'LRB',
 'NNP',
 'PRP$',
 'UH',
 'JJS',
 'POS',
 'VB',
 ':',
 ',',
 'RB',
 'NN',
 'NNS',
 'WP$',
 '$',
 '.',
 'PRP',
 'FW',
 'WDT',
 'JJ',
 'EX',
 'RRB',
 'RP',
 'VBN',
 ';',
 'CC',
 'VBZ',
 'PDT',
 'VBP',
 'VBG',
 'MD',
 'RBS']

In [6]:
words = list(set(data["Word"].values))
words.append("DUMMY") # Add a dummy word to pad sentences.

In [7]:
# Code to read sentences

class ReadSentences(object): 
    
    def __init__(self, data):
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [8]:
sentences = ReadSentences(data).sentences # Read all sentences

In [9]:
# Convert words and tags into numbers
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}

In [10]:
# Prepare input and output data

from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 50
X = [[word2id[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=len(words)-1)
y = [[tag2id[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2id["."])

In [11]:
# Convert output to one-hot bit

from tensorflow.keras.utils import to_categorical
y = [to_categorical(i, num_classes=len(tags)) for i in y]

In [12]:
y[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
# Training and test split by sentences

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20)

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input

In [15]:
input = Input(shape=(max_len,)) # Input layer
model = Embedding(input_dim=len(words), output_dim=50, input_length=max_len)(input) # Word embedding layer
model = Dropout(0.1)(model) # Dropout
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model) # Bi-directional LSTM layer
out = TimeDistributed(Dense(len(tags), activation="softmax"))(model)  # softmax output layer

In [16]:
model = Model(input, out) # Complete model

In [17]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]) # Compile with an optimizer

In [19]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=3, validation_split=0.1, verbose=1) # Train

Epoch 1/3
1080/1080 [==============================] - 290s 248ms/step - loss: 0.5768 - accuracy: 0.8395 - val_loss: 0.1374 - val_accuracy: 0.9607
Epoch 2/3
1080/1080 [==============================] - 246s 228ms/step - loss: 0.0897 - accuracy: 0.9754 - val_loss: 0.0707 - val_accuracy: 0.9795
Epoch 3/3
1080/1080 [==============================] - 286s 265ms/step - loss: 0.0523 - accuracy: 0.9855 - val_loss: 0.0595 - val_accuracy: 0.9827


In [20]:
# Demo test on one sample. See how it is mostly correct, but not 100%

i = 1213 # Some test sentence sample
p = model.predict(np.array([X_te[i]])) # Predict on it
p = np.argmax(p, axis=-1) # Map softmax back to a POS index
for w, pred in zip(X_te[i], p[0]): # for every word in the sentence
    print("{:20} -- {}".format(words[w], tags[pred])) # Print word and tag

1/1 [==============================] - 2s 2s/step
Mr.                  -- NNP
Netanyahu            -- NNP
was                  -- VBD
elected              -- VBN
to                   -- TO
lead                 -- VB
Likud                -- NNP
last                 -- JJ
month                -- NN
,                    -- ,
after                -- IN
Mr.                  -- NNP
Sharon               -- NNP
left                 -- VBD
the                  -- DT
party                -- NN
.                    -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY            

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [22]:
from nltk import word_tokenize

sentence = nltk.word_tokenize('That was a nice jump')
X_Samp = pad_sequences(maxlen=max_len, sequences=[[word2id[word] for word in sentence]], padding="post", value=len(words)-1)

In [23]:
p = model.predict(np.array([X_Samp[0]])) # Predict on it
p = np.argmax(p, axis=-1) # Map softmax back to a POS index
for w, pred in zip(X_Samp[0], p[0]): # for every word in the sentence
    print("{:20} -- {}".format(words[w], tags[pred])) # Print word and tag

1/1 [==============================] - 0s 93ms/step
That                 -- DT
was                  -- VBD
a                    -- DT
nice                 -- JJ
jump                 -- NN
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY                -- .
DUMMY 